# I. Anneaux usuels

## 1. Entiers et entiers modulaires

SageMath possède plusieurs types d'entiers. Le plus courant est le type `Integer`, qui est celui d'un entier entré directement au clavier. L'anneau $\mathbb Z$ correspondant s'obtient avec la commande `ZZ`.

In [ ]:
x=3^7654
print(type(x),type(0),type(-1))
ZZ

Attention que certaines commandes nécessitent ou renvoient des entiers d'un autre type, le type `int` natif de Python. On peut avoir besoin de conversions entre ces deux types. 

In [ ]:
x=ZZ.random_element(1000)
y=randrange(1000)
print(x.bits())
print(y.bits())

On peut récupérer assez facilement une écriture en base $B$ et inversement :

In [ ]:
x=3465457
L=x.digits(10)
print(L)
ZZ(L,10)

Notons que les corps $\mathbb Q$ et $\mathbb R$ s'obtiennent avec les commandes `QQ` et `RR`.

In [ ]:
ZZ.fraction_field() == QQ

Pour créer l'anneau $\mathbb Z/n \mathbb Z$, la commande la plus simple est `Zmod(n)`.

In [ ]:
A=Zmod(101)
x=A.random_element()
print(type(x))
print(parent(x))

En utilisant la complétion automatique, déterminer :
- l'ordre de 78 dans le groupe $(\mathbb Z/101 \mathbb Z)^*$
- un générateur de ce groupe multiplicatif

In [ ]:
A(78).multiplicative_order()

Par défaut, les éléments de $\mathbb Z/n \mathbb Z$ sont représentés entre $0$ et $n-1$. On peut expliciter un relevé avec `lift` ou `lift_centered`.
Attention que SageMath est un peu trop souple sur les conversions automatiques...

In [ ]:
A=Zmod(101)
B=Zmod(70)
x=A(76)
print(B(x))
print(B(x.lift_centered()))

En pratique il faut souvent choisir si on travaille avec des entiers ou avec des classes d'équivalence modulo $n$ 

**Exercice :** écrire deux programmes d'exponentiation rapide qui prennent en argument trois entiers $a$, $b$ et $n$ et renvoient le reste de la division euclidienne de $a^b$ par $n$. Le premier programme ne doit manipuler que des entiers, tandis que le deuxième doit utiliser $\mathbb Z/n\mathbb Z$.  

## 2. Anneaux de polynômes

La syntaxe pour créer un anneau de polynômes `A` en une ou plusieurs variables sur un anneau `R` est `A.<x,y,z> = R[]` ou `A.<x,y,z> = PolynomialRing(R)` (cette seconde version permet des options éventuelles). Notons que `R` peut être lui-même un anneau de polynômes.

In [ ]:
A.<x,y> = PolynomialRing(ZZ)
P=x^2*y^2 + x*y^2 + x*y + y + 1
print(A)
print(P)

B.<x> =  PolynomialRing(ZZ)
A2.<y> = PolynomialRing(B)
Q=x^2*y^2 + x*y^2 + x*y + y + 1
print(A2)
print(Q)


Attention à ne pas écraser les inconnues ! Il faut si nécessaire ré-exécuter une cellule précédente. On peut cependant récupérer les indéterminées avec la commande `A.gens()`

In [ ]:
print(parent(y^2+x^3+x+1))
y=3
print(y^2+x^3+x+1)
print(A.gens()[1])

Pour le calcul symbolique on peut souvent se contenter de déclarer des variables, au lieu d'un anneau de polynômes. Mais on n'a pas alors accès aux mêmes méthodes.

In [ ]:
A.<z>=PolynomialRing(QQ)
print((z^3+z-1)*(z^5-2*z+2))
print((z^2-1).xgcd(z^2-2*z+1))

var('z')
print((z^3+z-1)*(z^5-2*z+2))
print((z^2-1).xgcd(z^2-2*z+1))

Beaucoup de commandes ont des noms assez transparents : `P.coefficients()`, `P.degree()`, `P.derivative()`, etc.

**Exercices :** 
- récupérer le quotient et le reste de la division euclidienne de deux polynômes univariés
- tester si le polynôme nul a un coefficient dominant
- évaluer un polynôme
- récupérer le coefficient en $x^2$ d'un polynôme de $\mathbb Q[x]$.

La commande `P.base_extend(B)` permet d'agrandir facilement l'anneau des coefficients de $P$, accessible par la commande `P.base_ring()`.

Exemple : le code ci-dessous ne donne pas vraiment le résultat attendu. Comment modifier la dernière instruction pour obtenir un résultat qui nous convient ?

In [ ]:
A.<x> = PolynomialRing(ZZ)
P=x^5+x^3+2 
Q=2*x^2-1

P // Q

# II. Multiplication rapide de polynômes

## 1. Multiplication standard

**Exercice :** Implémenter l'algorithme standard de multiplication de deux polynômes $P$ et $Q$. On n'autorise que la somme ou la différence de deux polynômes et la multiplication par $X^n$ (décalage), ainsi bien sûr que l'accès aux coefficients et au degré.

Tester cet algorithme et s'assurer de la correction du résultat.

## 2. Karatsuba

L'algorithme de Karatsuba accélère le produit en utilisant une technique type "diviser pour régner". L'idée est la suivante :
* Si $P$ et/ou $Q$ sont de degré inférieur à un certain seuil, on effectue le produit standard.
* Sinon, on pose $m=\lfloor n/2\rfloor$ avec $n=\max(\deg(P),\deg(Q))$. On écrit la division euclidienne
  $$P = X^m P_1 + P_0 \quad \mathrm{et} \quad Q = X^m Q_1+Q_0.$$
  On a alors
  \begin{align*} PQ &= X^{2m} P_1Q_1 + X^{m}(P_1Q_0+P_0Q_1) + P_0Q_0 \\
  &= X^{2m} P_1Q_1 + X^{m}\left((P_1+P_0)(Q_1+Q_0) - P_1Q_1-P_0Q_0\right) + P_0Q_0 \end{align*}
  de telle sorte que l'on n'a à calculer que trois produits de polynômes de taille moitié : $P_0Q_0$, $P_1Q_1$, et $(P_1+P_0)(Q_1+Q_0)$.
  Chacun de ces trois sous-produits se calcule par un appel récursif à l'algorithme de Karatsuba.

**Exercice :** Écrire une fonction `karatsuba(P,Q,seuil)` qui multiplie deux polynômes par l'algorithme de Karatsuba. Comme précédemment, on s'autorise à utiliser la somme ou la différence de deux polynômes et la multiplication par une puissance de $X$. 

Pour la division euclidienne d'un polynôme $P$ par $X^m$, ce qui revient juste à tronquer la liste de ses coefficients, on pourra utiliser la conversion en polynôme de `list(P)[:m]` et `list(P)[m:]`.

Tester cet algorithme et s'assurer de la correction du résultat.

En prenant un seuil de $8$, comparer les temps d'exécution de l'algorithme standard et de celui de Karatsuba pour des polynômes de degré $d$ entre $1$ et $200$. On prendra un corps fini (par exemple $\mathbb{F}_{65537}$) comme anneau de base.

Présenter le résultat sous forme de graphe. 

## 3. Transformée de Fourier rapide

Pour aller asymptotiquement plus vite que Karatsuba, on utilise la transformée de Fourier discrète. On rappelle que dans l'algorithme de Cooley-Tukey, pour calculer la transformée de $u=(u_0,u_1,\dots,u_{n-1})$ avec $n$ une puissance de $2$ :
- on sépare $u$ en parties paire $v = (u_0,u_2,\dots,u_{n-2})$ et impaire $w=(u_1,u_3,\dots,u_{n-1})$
- on calcule récursivement leurs transformées $\hat v$ et $\hat w$
- on combine avec la formule $\hat u_\ell = \hat v_\ell + \omega^\ell \hat w_\ell$ pour tout $0\leq \ell \leq n-1$, où $\omega = e^{-\frac{2i\pi}{n}}$ et les indices sont pris modulo $n/2$ pour $\hat v$ et $\hat w$.


Programmer l'algorithme de Cooley-Tukey. Écrire aussi un programme calculant la transformée de Fourier discrète inverse. Les tester. 

Utiliser les programmes ci-dessus pour implémenter un algorithme de multiplication rapide dans $\mathbb C[X]$, que l'on testera.

Comparer les temps d'exécution de l'algorithme standard et de celui de Karatsuba pour des polynômes aléatoires de $\mathbb C[X]$ de degré $d$ entre $500$ et $2000$ (la méthode `random_element(d)` fonctionne). 